In [4]:
import pandas as pd
import numpy as np

import pickle
import random
import os

import matplotlib.pyplot as plt
import seaborn as sns

import networkx as nx
from stellargraph import StellarGraph, StellarDiGraph

In [5]:
import sys
sys.path.append('src')

from subsamplingmethods import *

***Loading the data***

In [46]:
# Load DataFrame 
df = pd.read_csv(r'../data/trans_3w.csv')
print(df.shape)
df.head()

(12529162, 9)


,txn_hash,input_address,output_address,ammount,fees,block_index,block_time,input_flag,output_flag
0,a71d37526455e709bef7b57f01ac65102d0f6584db0985...,136v9qFHYFbCeKjA6UFpTNriSAiiJqTjXw,1FkPDNR6V72AkRtvmHtM9BRvSADVwUFjxf,2.507314e+08,100000.000000,455974.0,2017-03-06 06:37:37 +0000 UTC,0,0
1,7025bd50d69cc79ca4ef71b8faa9a99cbb48c229c3fbf5...,1Mk2xL5DzJAS5EHfMmQ6wjQhEHxGsZBUZQ,1FbmDx9siau3V7wLB36z96qFVX2yEowQs,4.591702e+04,1169.294129,455974.0,2017-03-06 06:37:37 +0000 UTC,0,0
2,7025bd50d69cc79ca4ef71b8faa9a99cbb48c229c3fbf5...,1Hk8eK6SzuKtZCjdn2MU2nNUH4Sx8nXs1v,1FbmDx9siau3V7wLB36z96qFVX2yEowQs,5.511830e+05,14036.082391,455974.0,2017-03-06 06:37:37 +0000 UTC,0,0
3,7025bd50d69cc79ca4ef71b8faa9a99cbb48c229c3fbf5...,1Mk2xL5DzJAS5EHfMmQ6wjQhEHxGsZBUZQ,1PNv2u3MHVUJt158cYeUbUoifovKu9aKZL,2.714023e+05,6911.362532,455974.0,2017-03-06 06:37:37 +0000 UTC,0,0
4,7025bd50d69cc79ca4ef71b8faa9a99cbb48c229c3fbf5...,1Hk8eK6SzuKtZCjdn2MU2nNUH4Sx8nXs1v,1PNv2u3MHVUJt158cYeUbUoifovKu9aKZL,3.257885e+06,82963.260948,455974.0,2017-03-06 06:37:37 +0000 UTC,0,0


In [4]:
df.tail()

,txn_hash,input_address,output_address,ammount,fees,block_index,block_time,input_flag,output_flag
12529157,593e582e2955761fe88214a2e6e17026429c1c9e93d3f3...,1FMGQrLXHzYNBfkrpA37cCvso8239PJevH,3Fp5ejYY8FsJ6Y3kb377VRjJunTeUVYsuq,77469.636654,29374.363346,453359.0,2017-02-16 20:38:27 +0000 UTC,0,0
12529158,593e582e2955761fe88214a2e6e17026429c1c9e93d3f3...,1JwqgvGBKZf7X6ZYxc9NMBzUxo7tnHcx8K,3Fp5ejYY8FsJ6Y3kb377VRjJunTeUVYsuq,77635.678236,29437.321764,453359.0,2017-02-16 20:38:27 +0000 UTC,0,0
12529159,593e582e2955761fe88214a2e6e17026429c1c9e93d3f3...,17iUzMZVkx4tcUCQ3nrTUkZ1ELPYGhRANe,3Fp5ejYY8FsJ6Y3kb377VRjJunTeUVYsuq,78029.392555,29586.607445,453359.0,2017-02-16 20:38:27 +0000 UTC,0,0
12529160,593e582e2955761fe88214a2e6e17026429c1c9e93d3f3...,125q65xjFfTzXoYLtzv6fVvSFCRQUgYV9e,3Fp5ejYY8FsJ6Y3kb377VRjJunTeUVYsuq,78307.820361,29692.179639,453359.0,2017-02-16 20:38:27 +0000 UTC,0,0
12529161,593e582e2955761fe88214a2e6e17026429c1c9e93d3f3...,1E33Ti9DQKmuZ72MEzDe2hBj4WiPYndgxB,3Fp5ejYY8FsJ6Y3kb377VRjJunTeUVYsuq,78842.198727,29894.801273,453359.0,2017-02-16 20:38:27 +0000 UTC,0,0


In [47]:
# Time stap 1,2,3,4
df['timestamp'] = (df['block_index'] - df['block_index'].min() + 1).astype(int)
df['timestamp'] = df['timestamp'].astype(str)

***Creation of the Graph***

In [48]:
# Rename some features
new_var_names = {'input_address': 'source', 'output_address': 'target', 'ammount': 'weight'}
df.rename(new_var_names, axis=1, inplace=True)

In [49]:
# Graph
G = StellarDiGraph(edges=df[['source', 'target', 'weight']])
print(G.info())

StellarDiGraph: Directed multigraph
 Nodes: 4655451, Edges: 12529162

 Node types:
  default: [4655451]
    Features: none
    Edge types: default-default->default

 Edge types:
    default-default->default: [12529162]
        Weights: range=[9.15703e-05, 2.75399e+11], mean=2.92531e+07, std=2.95969e+08
        Features: none


***Largest weakly connected component***

In [8]:
w_c_c = max(G.connected_components(), key=len) # weakly largest connected component
print(len(w_c_c))

3827352


In [50]:
G = G.subgraph(w_c_c)
print(G.info())

StellarDiGraph: Directed multigraph
 Nodes: 3827352, Edges: 11724249

 Node types:
  default: [3827352]
    Features: none
    Edge types: default-default->default

 Edge types:
    default-default->default: [11724249]
        Weights: range=[9.15703e-05, 2.68337e+11], mean=2.72106e+07, std=2.78826e+08
        Features: none


***List of fraudulent and non-fraudulent nodes***

In [ ]:
# Fraudulent
output_labels = pickle.load(open("../../bse_clovrlabs_btc_fraud/data/output_labels.txt",'rb'))
input_labels = pickle.load(open("../../bse_clovrlabs_btc_fraud/data/input_labels.txt",'rb'))
fraudulent_nodes = list(set(list(output_labels) + list(input_labels)))
fraudulent_nodes = [node for node in fraudulent_nodes if node in G.nodes()]
print(len(fraudulent_nodes))

In [11]:
# Licits
licit_nodes = [node for node in G.nodes() if node not in fraudulent_nodes]
print(len(licit_nodes))
print(len(licit_nodes) + len(fraudulent_nodes) - len(G.nodes()))

3822926
0


***Data frame with information of weight and timestamp to add in the sampled graph***

In [51]:
# Dataframe only with the info of source, target, ammount (weight) and timestamp
df_f = df[['source', 'target', 'weight', 'timestamp']].drop_duplicates(subset=['source', 'target', 'weight'])
df_f.head()

,source,target,weight,timestamp
0,136v9qFHYFbCeKjA6UFpTNriSAiiJqTjXw,1FkPDNR6V72AkRtvmHtM9BRvSADVwUFjxf,2.507314e+08,2657
1,1Mk2xL5DzJAS5EHfMmQ6wjQhEHxGsZBUZQ,1FbmDx9siau3V7wLB36z96qFVX2yEowQs,4.591702e+04,2657
2,1Hk8eK6SzuKtZCjdn2MU2nNUH4Sx8nXs1v,1FbmDx9siau3V7wLB36z96qFVX2yEowQs,5.511830e+05,2657
3,1Mk2xL5DzJAS5EHfMmQ6wjQhEHxGsZBUZQ,1PNv2u3MHVUJt158cYeUbUoifovKu9aKZL,2.714023e+05,2657
4,1Hk8eK6SzuKtZCjdn2MU2nNUH4Sx8nXs1v,1PNv2u3MHVUJt158cYeUbUoifovKu9aKZL,3.257885e+06,2657


In [52]:
# Function to include additional info on the sample dataframe of the graph
def create_df(g_sampled, tot_df):
    g_sampled_nx = StellarDiGraph.to_networkx(g_sampled)
    g_sampled_df = nx.to_pandas_edgelist(g_sampled_nx)
    g_sampled_df = g_sampled_df.drop_duplicates()
    
    df_merged = pd.merge(g_sampled_df, tot_df, on = ['source', 'target', 'weight'])
    
    return df_merged

***Sampled graphs using the random walk method (RWS)***

In [14]:
rw_results = []

for iter_ in range(9,10):
    g_sampled = rw_sampling(G, 50000) # Sample graph
    df_ss = create_df(g_sampled, df_f)    # Create the dataframe of the graph 
    df_ss.to_csv('outputs/random_walk/g_ss_rw_' + str(iter_) + '.csv', index=False) # Export the graph
    
    rw_results.append({ 'iter' : iter_,
                        'nodes' : StellarDiGraph.number_of_nodes(g_sampled) ,
                        'edges' : df_ss.shape[0],
                        'high_risky_nodes' : len([node for node in g_sampled.nodes() if node in fraudulent_nodes])
                        })


In [31]:
df_results_rw = pd.DataFrame(rw_results)
df_results_rw['hr_percentage'] = (df_results_rw['high_risky_nodes']/df_results_rw['nodes']) * 100
df_results_rw

,iter,nodes,edges,high_risky_nodes,hr_percentage
0,0,50000,194318,328,0.656000
1,1,50000,193632,296,0.592000
2,2,50000,189581,306,0.612000
3,3,50000,192088,308,0.616000
4,4,49999,191702,332,0.664013
5,5,50000,189397,291,0.582000
6,6,50000,200436,318,0.636000
7,7,50000,193629,324,0.648000
8,8,50000,192785,294,0.588000
9,9,49999,194299,330,0.660013


***Sampled graphs using the neighbors method (NS)***

In [24]:
ne_results = []

for iter_ in range(1,10):
    g_sampled = neighbors_sampling(G, fraudulent_nodes, licit_nodes, 600) # Sample graph
    df_ss = create_df(g_sampled, df_f)    # Create the dataframe of the graph 
    df_ss.to_csv('outputs/neighbors_method/g_ss_ne_' + str(iter_) + '.csv', index=False) # Export the graph
    
    ne_results.append({ 'iter' : iter_,
                        'nodes' : StellarDiGraph.number_of_nodes(g_sampled) ,
                        'edges' : df_ss.shape[0],
                        'high_risky_nodes' : len([node for node in g_sampled.nodes() if node in fraudulent_nodes])
                        })


In [25]:
df_results_ne = pd.DataFrame(ne_results)
df_results_ne['hr_percentage'] = (df_results_ne['high_risky_nodes']/df_results_ne['nodes']) * 100
df_results_ne

,iter,nodes,edges,high_risky_nodes,hr_percentage
0,1,36424,181043,1065,2.923896
1,2,56508,221820,1117,1.976711
2,3,51941,309286,1129,2.173620
3,4,57302,255240,1131,1.973753
4,5,79483,608876,1255,1.578954
5,6,51758,288132,1125,2.173577
6,7,49052,242306,1071,2.183397
7,8,36108,228745,1181,3.270743
8,9,60932,456214,1239,2.033414


***Sampled graphs using the mixed  method (MS)***

In [ ]:
mix_results = []

for iter_ in range(10):
    g_sampled = mixed_sampling(G, fraudulent_nodes, 50000)
    df_ss = create_df(g_sampled, df_f)    # Create the dataframe of the graph 
    df_ss.to_csv('outputs/mixed_method/g_as_mm_' + str(iter_) + '.csv', index=False) # Export the graph
    
    mix_results.append({ 'iter' : iter_,
                        'nodes' : StellarDiGraph.number_of_nodes(g_sampled) ,
                        'edges' : df_ss.shape[0],
                        'high_risky_nodes' : len([node for node in g_sampled.nodes() if node in fraudulent_nodes])
                        })


In [19]:
df_results_mm = pd.DataFrame(mix_results)
df_results_mm['hr_percentage'] = (df_results_mm['high_risky_nodes']/df_results_mm['nodes']) * 100
df_results_mm

,iter,nodes,edges,high_risky_nodes,hr_percentage
0,0,50000,202230,4343,8.686
1,1,50000,196727,4342,8.684
2,2,50000,196479,4342,8.684
3,3,50000,198682,4342,8.684
4,4,50000,196084,4342,8.684
5,5,50000,199006,4342,8.684
6,6,50000,190156,4342,8.684
7,7,50000,197911,4342,8.684
8,8,50000,201021,4342,8.684
9,9,50000,194127,4342,8.684


***Sampled graphs using the random nodes selection method (CS)***

In [53]:
# This needs a little bit more preprocessing

# Bring the information of eigen centrality
node_properties = pd.read_csv('../../o_data/node_properties_cl.csv')
node_properties = node_properties[node_properties['address'].isin(list(w_c_c[0]))]
print(node_properties.shape)

(3827352, 6)


In [16]:
# Remove nodes with low centrality from the list of licit nodes
licit_nodes_e = node_properties[node_properties['fraud_flag'] == 0] # sample from low-risk nodes
licit_nodes_e = licit_nodes_e[licit_nodes_e['eigen_centrality'] > 1.818528e-6] # remove nodes in bottom 84% of centrality
licit_nodes_e = licit_nodes_e['address'].tolist()
len(licit_nodes_e)

362814

In [18]:
rn_results = []

for iter_ in range(10):
    g_sampler = RandomNodeSampler(G, fraudulent_nodes, licit_nodes_e, 50000)
    g_sampled = g_sampler.sample(G, fraudulent_nodes, licit_nodes, 1500) # Sample graph
    df_ss = create_df(g_sampled, df_f)    # Create the dataframe of the graph 
    df_ss.to_csv('outputs/random_nodes/g_ss_rn_' + str(iter_) + '.csv', index=False) # Export the graph
    
    rn_results.append({ 'iter' : iter_,
                        'nodes' : StellarDiGraph.number_of_nodes(g_sampled) ,
                        'edges' : df_ss.shape[0],
                        'high_risky_nodes' : len([node for node in g_sampled.nodes() if node in fraudulent_nodes])
                        })


In [19]:
df_results_rn = pd.DataFrame(rn_results)
df_results_rn['hr_percentage'] = (df_results_rn['high_risky_nodes']/df_results_rn['nodes']) * 100
df_results_rn

,iter,nodes,edges,high_risky_nodes,hr_percentage
0,0,50000,116671,4426,8.852
1,1,50000,111102,4426,8.852
2,2,50000,111514,4426,8.852
3,3,50000,117039,4426,8.852
4,4,50000,111077,4426,8.852
5,5,50000,110894,4426,8.852
6,6,50000,117004,4426,8.852
7,7,50000,108538,4426,8.852
8,8,50000,114743,4426,8.852
9,9,50000,109641,4426,8.852


***Sampling the graph using the degree biased Random Sampler (DS)***

In [54]:
# Obtain sub-graph with short-listed nodes
G_licit = G.subgraph(licit_nodes)

In [96]:
# Obtain probabilites
dp = node_properties[node_properties['address'].isin(licit_nodes)].copy()
dp['degree4'] = dp['degree']**4
dp_sum = dp['degree4'].sum()
dp['p'] = dp['degree4'] / dp_sum
p_degree = dp['p'].values
print(sum(p_degree))

0.9999999999528445


In [ ]:
bd_results = []

for iter_ in range(3):
    print('Doing:  ', iter_, '...')
    g_sampler = DegreeBasedSampler(G, G_licit, fraudulent_nodes, licit_nodes, 50000, p_degree) 
    g_sampled = g_sampler.sample(G, G_licit, fraudulent_nodes, licit_nodes, 50000, p_degree) 
    df_ss = create_df(g_sampled, df_f)    # Create the dataframe of the graph 
    df_ss.to_csv('outputs/biased_degree/g_ss_bd_' + str(iter_) + '.csv', index=False) # Export the graph
    
    bd_results.append({ 'iter' : iter_,
                        'nodes' : len(set(list(df_ss['source'].unique()) + list(df_ss['target'].unique()))),
                        'edges' : df_ss.shape[0],
                        'high_risky_nodes' : len([node for node in g_sampled.nodes() if node in fraudulent_nodes])
                        })


### Table with a summary of the sample methods


In [7]:
output_labels = pickle.load(open("../../bse_clovrlabs_btc_fraud/data/output_labels.txt",'rb'))
input_labels = pickle.load(open("../../bse_clovrlabs_btc_fraud/data/input_labels.txt",'rb'))
fraudulent_nodes = list(set(list(output_labels) + list(input_labels)))

In [8]:
rn = ['outputs/random_nodes/' + name for name in os.listdir('outputs/random_nodes/') if name.endswith('csv')]
mm = ['outputs/mixed_method/' + name for name in os.listdir('outputs/mixed_method/') if name.endswith('csv')]
rw = ['outputs/random_walk/' + name for name in os.listdir('outputs/random_walk/') if name.endswith('csv')]
nm = ['outputs/neighbors_method/' + name for name in os.listdir('outputs/neighbors_method/') if name.endswith('csv')]
db = ['outputs/biased_degree/' + name for name in os.listdir('outputs/biased_degree/') if name.endswith('csv')]

In [9]:
all_ = rn + mm + rw + nm + db

In [19]:
summary_ = []
for file in all_:
    df_f = pd.read_csv(file)
    nodes = list(set(list(df_f['source'].unique()) + list(df_f['target'].unique())))
    hr_nodes = len([node for node in nodes if node in fraudulent_nodes])
    summary_.append({   'file' : file,
                        'nodes' : len(nodes),
                        'edges' : df_f.shape[0],
                        'high_risk_nodes' : hr_nodes,
                        'high_risk_ratio' : hr_nodes * 100 /len(nodes)
                        })

In [23]:
df_sum = pd.DataFrame(summary_)
print(df_sum.shape)
df_sum.sample(10)

(50, 5)


,file,nodes,edges,high_risk_nodes,high_risk_ratio
18,outputs/mixed_method/g_as_mm_0.csv,49985,202230,4333,8.668601
35,outputs/neighbors_method/g_ss_ne_7.csv,49052,242306,1071,2.183397
39,outputs/neighbors_method/g_ss_ne_4.csv,57302,255240,1131,1.973753
32,outputs/neighbors_method/g_ss_ne_1.csv,36424,181043,1065,2.923896
34,outputs/neighbors_method/g_ss_ne_5.csv,79483,608876,1255,1.578954
25,outputs/random_walk/g_ss_rw_4.csv,50000,189397,291,0.582000
28,outputs/random_walk/g_ss_rw_7.csv,50000,192785,294,0.588000
7,outputs/random_nodes/g_ss_rn_9.csv,29244,119883,2252,7.700725
17,outputs/mixed_method/g_as_mm_1.csv,50000,196727,4353,8.706000
22,outputs/random_walk/g_ss_rw_8.csv,50000,189581,306,0.612000


In [29]:
df_sum['Sampling_method'] = df_sum['file'].apply(lambda x: x.split('/')[1])
df_sum['Sampling_method'].value_counts()

biased_degree       10
neighbors_method    10
random_nodes        10
random_walk         10
mixed_method        10
Name: Sampling_method, dtype: int64

In [40]:
df_sum_avg = df_sum.groupby(['Sampling_method']).mean()
df_sum_avg[['nodes', 'edges', 'high_risk_nodes']] = df_sum_avg[['nodes', 'edges', 'high_risk_nodes']].round().astype(int)
df_sum_avg['high_risk_ratio'] = df_sum_avg['high_risk_ratio'].round(2) 
df_sum_avg

,nodes,edges,high_risk_nodes,high_risk_ratio
Sampling_method,,,,
biased_degree,30499,316968,2585,8.48
mixed_method,49998,197242,4351,8.70
neighbors_method,55770,322565,1169,2.21
random_nodes,29487,122282,2251,7.63
random_walk,50000,193187,313,0.63


In [38]:
df_sum_avg.to_csv('outputs/tables/sampled_graphs_results.csv')